In [ ]:
pwd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
def RACIPE_data_import(filename):
  Network = pd.read_table(filename[:-13] + '_solution.dat', sep ='\t', index_col  = False, header = None) 
  Network_prs= pd.read_csv(filename[:-13] + '.prs', sep ='\t', index_col  = False) 

  c = ['Model Number','Stable States','Number of Runs that converged']
  p = 'P'
  i = 0

  while p =='P':
    c.append(Network_prs.iloc[i,0][8:])
    p = Network_prs.iloc[i+1,0][0]
    i = i + 1
  Network.columns= c
  return Network



In [ ]:
Network_3_T1 = RACIPE_data_import('Network5_trial1_solution.dat')
Network_3_T2 = RACIPE_data_import('Network5_trial2_solution.dat')
Network_3_T3 = RACIPE_data_import('Network5_trial3_solution.dat')

In [ ]:
def stable_states(Network_data):
    Network_3_SS= pd.DataFrame([[0]]*10)
    Network_3_SS.index=['Monostable','Bistable','Tristable','Tetrastable','5 Stable States', '6 Stable States', '7 Stable States', '8 Stable States', '9 Stable States', '10 Stable States']
    Network_3_SS.columns = ['Cases']

    Network_3_SS.iloc[:,0] = np.array(Network_data.iloc[:,1].value_counts().sort_index())
    Network_3_SS

    for i in range(0,10):
        Network_3_SS.iloc[i,0] = Network_3_SS.iloc[i,0]/(i+1)
    return Network_3_SS


In [ ]:
Network_3_SS_T1= stable_states(Network_3_T1)
Network_3_SS_T2= stable_states(Network_3_T2)
Network_3_SS_T3= stable_states(Network_3_T3)

In [ ]:
SS_summary_table_network3=Network_3_SS_T1.copy()
SS_summary_table_network3.loc[:,'Trial 1'] = Network_3_SS_T1.iloc[:,0]
SS_summary_table_network3.loc[:,'Trial 2'] = Network_3_SS_T2.iloc[:,0]
SS_summary_table_network3.loc[:,'Trial 3'] = Network_3_SS_T3.iloc[:,0]
SS_summary_table_network3=SS_summary_table_network3.drop(columns=['Cases'])
SS_summary_table_network3=SS_summary_table_network3/100

In [ ]:
SS_summary_table_network3.loc[:,'Mean'] = SS_summary_table_network3.mean(axis=1)
SS_summary_table_network3.loc[:,'Standard Deviation'] = SS_summary_table_network3.std(axis=1)
SS_summary_table_network3

In [ ]:
SS_summary_table_network3=SS_summary_table_network3.round(2)
SS_summary_table_network3.loc[:,'labels'] = SS_summary_table_network3.loc[:,'Mean'].astype(str) + ' ± ' + SS_summary_table_network3.loc[:,'Standard Deviation'].astype(str) + '%'
SS_summary_table_network3

Figure 4 A

In [ ]:
plt.figure(figsize=(6, 10))
plt.rcParams['figure.dpi']= 1000
colors = sns.color_palette('pastel')[0:10]
patches, texts = plt.pie(SS_summary_table_network3.loc[:,'Mean'], labels = SS_summary_table_network3.loc[:,'labels'], colors = colors,
                         
                         textprops={"fontsize":16} )



#plt.legend(patches, SS_summary_table_network3.index, loc='upper left' )
plt.show()

In [ ]:
plt.figure(figsize=(6, 10))
plt.rcParams['figure.dpi']= 1000
colors = sns.color_palette('pastel')[0:10]

#patches, texts = plt.pie(SS_summary_table_network3.loc[:,'Mean'], labels = SS_summary_table_network3.loc[:,'labels'], colors = colors)
plt.legend(patches, SS_summary_table_network3.index, loc='upper left' )
plt.show()

EM and SN Plots

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
def standard_scaling(dataframe):
    cols = dataframe.columns 
    return pd.DataFrame(ss.fit_transform(dataframe), columns = cols)

In [ ]:
Network_3_T1.loc[:,'ERa66':] = standard_scaling(Network_3_T1.loc[:,'ERa66':])
Network_3_T2.loc[:,'ERa66':] = standard_scaling(Network_3_T2.loc[:,'ERa66':])
Network_3_T3.loc[:,'ERa66':] = standard_scaling(Network_3_T3.loc[:,'ERa66':])
Network_3_T1

In [ ]:
def EM_SN_scorer(Network):
    Network.loc[:,'SN Score'] = (Network.loc[:,'OCT4'] + Network.loc[:,'LIN28'] - Network.loc[:,'miR145'] - Network.loc[:,'let7'])/4
    Network.loc[:,'EM Score'] = (Network.loc[:,'ZEB'] + Network.loc[:,'SLUG'] - Network.loc[:,'miR200'] - Network.loc[:,'CDH1'])/4



In [ ]:
EM_SN_scorer(Network_3_T1)
EM_SN_scorer(Network_3_T2)
EM_SN_scorer(Network_3_T3)

In [ ]:
Network_3_T1

In [ ]:
sns.set(font_scale=1.75)
plt.rcParams['figure.dpi']= 500
sns.clustermap(Network_3_T1.loc[:,'ERa66':'OCT4'], metric="euclidean", cmap ='seismic', standard_scale=1, method="ward")

In [ ]:
'''sns.set(font_scale=4)
sns.clustermap(Network_3_T1.loc[:,'ERa66':'OCT4'], metric="euclidean", cmap ='seismic', standard_scale=1, method="ward")'''

In [ ]:
def EM_kde_plot(Network, EM_upper , EM_lower ):
    plt_1 = plt.figure(figsize=(2, 3))
    #sns.set_palette(palette)
    sns.displot(Network, x="EM Score", fill=True, bins = 120 ,kde = True, color = 'red')

    plt.axvline(x = EM_lower, color = 'black', linestyle = ':')
    plt.axvline(x = EM_upper, color = 'black', linestyle = ':')
    plt.xlabel('EM Score')
    plt.ylabel('Frequency')

In [ ]:
def SN_kde_plot(Network, SN_upper , SN_lower ):
    plt_1 = plt.figure(figsize=(2, 3))
    #sns.set_palette(palette)
    sns.displot(Network, x="SN Score", fill=True, bins = 120 ,kde = True, color = 'purple')
    plt.axvline(x = SN_lower, color = 'r', linestyle = ':')
    plt.axvline(x = SN_upper, color = 'r', linestyle = ':')
    plt.xlabel('SN Score')
    plt.ylabel('Frequency')


In [ ]:
def EM_SN_classifier(Network, EM_upper, EM_lower, SN_upper, SN_lower):
    for i in range(len(Network)):
        if Network.loc[i,'EM Score'] >= EM_upper :
            Network.loc[i,'EM_Status'] = 'Mesenchymal'

        elif Network.loc[i,'EM Score'] < EM_lower :
            Network.loc[i,'EM_Status'] = 'Epithelial'
        else :
            Network.loc[i,'EM_Status'] = 'Hybrid'
        
        
        if Network.loc[i,'SN Score'] >= SN_upper :
            Network.loc[i,'Stemness_Status'] = 'Non-Stem-like'

        elif Network.loc[i,'SN Score'] < SN_lower :
            Network.loc[i,'Stemness_Status'] = 'Non-Stem-like'
            
        else :
            Network.loc[i,'Stemness_Status'] = 'Stem-like'


    
        if Network.loc[i,'BACH1'] >= 0 :
            Network.loc[i,'BACH1_Status'] = 'BACH1_Positive'

        elif Network.loc[i,'BACH1'] < 0 :
            Network.loc[i,'BACH1_Status'] = 'BACH1_Negative'

        
        if Network.loc[i,'RKIP'] >= 0 :
            Network.loc[i,'RKIP_Status'] = 'RKIP_Positive'

        elif Network.loc[i,'RKIP'] < 0 :
            Network.loc[i,'RKIP_Status'] = 'RKIP_Negative'



In [ ]:
def EM_kde_plot(Network, EM_upper , EM_lower ):
    plt.figure(figsize=(20, 30))
    #sns.set_palette(palette)
    sns.displot(Network, x="EM Score", fill=True, bins = 120 ,kde = True, color = 'red')
    plt.rcParams['figure.dpi']= 500
    plt.axvline(x = EM_lower, color = 'black', linestyle = ':')
    plt.axvline(x = EM_upper, color = 'black', linestyle = ':')

    plt.xticks(fontsize = 14, fontname='Arial')
    plt.yticks(fontsize = 14, fontname='Arial')
    plt.xlabel('EM Score',fontsize = 14, fontname='Arial',  fontweight='bold')

    plt.ylabel('Frequency',fontsize = 14, fontname='Arial',  fontweight='bold')

In [ ]:
def SN_kde_plot(Network, SN_upper , SN_lower ):
    plt.figure(figsize=(20, 30))
    plt.rcParams['figure.dpi']= 1000

    #sns.set_palette(palette)
    sns.displot(Network, x="SN Score", fill=True, bins = 120 ,kde = True, color = 'purple')
    plt.axvline(x = SN_lower, color = 'r', linestyle = ':')
    plt.axvline(x = SN_upper, color = 'r', linestyle = ':')
    #plt.xlabel('EM Score', )
    plt.xticks(fontsize = 14, fontname='Arial')
    plt.yticks(fontsize = 14, fontname='Arial')
    plt.xlabel('SN Score',fontsize = 14, fontname='Arial', fontweight='bold')
    plt.ylabel('Frequency', fontsize = 14, fontname='Arial', fontweight='bold')


In [ ]:
SN_kde_plot(Network_3_T1, 0.38 , -0.38 )
EM_kde_plot(Network_3_T1, 0.64 , -0.24)

In [ ]:

SN_kde_plot(Network_3_T2, 0.39 , -0.38 )
EM_kde_plot(Network_3_T2, 0.63 , -0.26)

In [ ]:
Network_3_T2

Figure S3B (i)

In [ ]:
from scipy.stats import kurtosis
from scipy.stats import skew


def BiC(dataframe, n, i):
    sk=skew(dataframe.loc[:,i],axis=0,bias=True)
    kur=kurtosis(dataframe.loc[:,i],bias=True,axis=0,fisher=True)

    ncum=(pow(n-1,2))/((n-2)*(n-3))
    bic=(pow(sk,2) + 1)/(kur +3*ncum)
    return bic

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#plt.rcParams['figure.dpi']= 500

# Set the style with blue background and white grid
sns.set_style("darkgrid")

# Figure size and layout
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(11, 7))

# Flatten the axes array for easier indexing
axes = axes.flatten()
plt.rcParams['figure.dpi']= 500
# Loop through each column and create histograms with thinner bars
#for i, column in enumerate(Network_3_T2.columns[3:15]):
#    sns.displot(Network_3_T2[column], ax=axes[i], kde=True )
    #axes[i].set_title(f'{column}', fontsize=12)
#    axes[i].set_xlabel(' ', fontsize=12)
#    axes[i].set_ylabel(' ', fontsize=12)
    #axes[i].text(0.05, 0.95, "BC " + str(round(BiC(Network_3_T2, 10000, column),2)), transform=axes[i].transAxes, fontsize=0.1, va='top', ha='left')

for i, column in enumerate(Network_3_T2.columns[3:15]):
    ax = axes[i]
    sns.histplot(Network_3_T2[column], ax=ax, kde=True,stat='probability', color='#1284c8')
    ax.set_xlabel(column, fontsize=12)
    ax.set_ylabel('', fontsize=12)
    ax.tick_params(axis='both', labelsize=10)
    ax.text(0.05, 0.95, "BC " + str(round(BiC(Network_3_T2, 10000, column),2)), transform=axes[i].transAxes, fontsize=12, va='top', ha='left')

    


  

fig.supxlabel('Z-Normalized Score', weight='bold',fontsize=16,x=0.55, y=0)
fig.supylabel('Kernal Density Estimate', weight='bold',fontsize=16,x=-0.0, y=0.55)

# Adjust the spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
#Re-indexing based on PC1 coefficients

corr_map=Network_3_T2.iloc[:,3:16].corr()
corr_map_reindex=corr_map.reindex(['ZEB', 'LIN28', 'SLUG', 'ERa36', 'OCT4', 'BACH1', 'RKIP', 'SNAIL',
       'ERa66', 'miR145', 'let7', 'miR200', 'CDH1'])
corr_map=corr_map_reindex.T
corr_map_reindex=corr_map.reindex(['ZEB', 'LIN28', 'SLUG', 'ERa36', 'OCT4', 'BACH1', 'RKIP', 'SNAIL',
       'ERa66', 'miR145', 'let7', 'miR200', 'CDH1'])
corr_map_reindex


Figure 3Bii

In [ ]:
# creating mask
plt.rcParams['figure.dpi']= 500
mask = np.triu(np.ones_like(corr_map_reindex, dtype=bool),k=0)
for i in range(len(mask)):
                mask[i][i]= False
sns.set_style("white")
# plotting a triangle correlation heatmap
dataplot = sns.heatmap(corr_map_reindex,cmap='coolwarm', annot=False, mask=mask,square=True, linecolor= 'white',linewidths=0.5,cbar_kws={'label': 'Spearman Coefficient (ρ)'})
cbar = dataplot.collections[0].colorbar
cbar.set_label('Spearman Coefficient (ρ)', labelpad=17)

# displaying heatmap
plt.show()

In [ ]:
Network_3_T2

PCA Analysis

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components =10)
pca.fit(Network_3_T2.loc[:,'ERa66':'OCT4'])

In [ ]:
PCA_data = pd.DataFrame(pca.components_[0:2],columns=Network_3_T2.loc[:,'ERa66':'OCT4'].columns).T.sort_values(by=0)
PCA_data.columns = ['PC1','PC2']
PCA_data

In [ ]:
from scipy.stats import spearmanr

In [ ]:
Network_3_T2.iloc[:,3:16].corr(method='spearman')

Checking p-value for each spearman co-relation

In [ ]:
corr_matrix, p_values = spearmanr(Network_3_T2.iloc[:,3:16])
p_val_matrix = pd.DataFrame(p_values, columns =Network_3_T2.iloc[:,3:16].columns, index = Network_3_T2.iloc[:,3:16].columns )
p_val_matrix

Figure 4C (i) & (ii)

In [ ]:

SN_kde_plot(Network_3_T3, 0.38 , -0.37 )
EM_kde_plot(Network_3_T3, 0.63 , -0.25)

Figure 4 D

In [ ]:
plt.figure(figsize=(7, 5))
#sns.set_style("whitegrid")
#plt.grid(False)
plt.rcParams['figure.dpi']= 1000
plo = sns.scatterplot(data=Network_3_T1, x="EM Score", y="SN Score", s = 2, color='Black' )
plt.xlabel("EM Score", fontweight='bold', size=17, fontname='Arial')
plt.ylabel('SN Score',  fontweight='bold' , size=17 , fontname='Arial')
plt.xticks(size=15)
plt.yticks(size=15)
#plo.plot(grid = False)
plt.axhline(y = -0.38, color = 'r', linestyle = ':')
plt.axhline(y = 0.38, color = 'r', linestyle = ':')
plt.axvline(x = -0.24, color = 'r', linestyle = ':')
plt.axvline(x = 0.64, color = 'r', linestyle = ':')




Spearman Coefficient EM vs SN Score

In [ ]:
spearmanr(Network_3_T1.loc[:,'EM Score'],Network_3_T1.loc[:,'SN Score'] )

In [ ]:
Network_3_T2

In [ ]:
SN_kde_plot(Network_3_T1, 0.38 , -0.38 )
EM_kde_plot(Network_3_T1, 0.64 , -0.24)

SN_kde_plot(Network_3_T2, 0.39 , -0.38 )
EM_kde_plot(Network_3_T2, 0.63 , -0.26)

SN_kde_plot(Network_3_T3, 0.38 , -0.37 )
EM_kde_plot(Network_3_T3, 0.63 , -0.25)

In [ ]:
EM_SN_classifier(Network_3_T1, 0.63 , -0.26, 0.39, -0.38)

In [ ]:
EM_SN_classifier(Network_3_T2, 0.63 , -0.26, 0.39 , -0.38 )

In [ ]:
EM_SN_classifier(Network_3_T3, 0.63, -0.25, 0.38 ,-0.37 )

In [ ]:
Network_3_T3.loc[:,'Stemness_Status'].value_counts()

Figure 4 E (i) 

In [ ]:
def summari(Network1,Network2,Network3,given, gene, status):
    dat1=pd.DataFrame(index=['Non-Stem-like','Stem-like'], columns=['Trial 1','Trial 2','Trial 3'])
    dat1.loc[:, 'Trial 1']=((Network1[Network1.loc[:, gene +'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network1[Network1.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Trial 2']= ((Network2[Network2.loc[:, gene+'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network2[Network2.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Trial 3']= ((Network3[Network3.loc[:, gene+'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network3[Network3.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Std']=dat1.loc[:,['Trial 1','Trial 2','Trial 3']].std(axis=1)
    dat1.loc[:,'Mean']=dat1.loc[:,['Trial 1','Trial 2','Trial 3']].mean(axis=1)
    dat1.loc[:,'Probabilty Given '+ given] = round(dat1.loc[:,'Mean']*100,2).astype(str) + ' ± ' + round(dat1.loc[:,'Std']*100,2).astype(str) + '%'
    return dat1

In [ ]:
def summari_rev(Network1,Network2,Network3,given, gene, status):
    dat1=pd.DataFrame(index=['BACH1_Positive','BACH1_Negative'], columns=['Trial 1','Trial 2','Trial 3'])
    dat1.loc[:, 'Trial 1']=((Network1[Network1.loc[:, gene +'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network1[Network1.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Trial 2']= ((Network2[Network2.loc[:, gene+'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network2[Network2.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Trial 3']= ((Network3[Network3.loc[:, gene+'_Status']==given]).loc[:, status].value_counts().sort_index())/sum((Network3[Network3.loc[:,  gene +'_Status']==given]).loc[:, status].value_counts().sort_index())
    dat1.loc[:,'Std']=dat1.loc[:,['Trial 1','Trial 2','Trial 3']].std(axis=1)
    dat1.loc[:,'Mean']=dat1.loc[:,['Trial 1','Trial 2','Trial 3']].mean(axis=1)
    dat1.loc[:,'Probabilty Given '+ given] = round(dat1.loc[:,'Mean']*100,2).astype(str) + ' ± ' + round(dat1.loc[:,'Std']*100,2).astype(str) + '%'
    return dat1

In [ ]:
(Network_3_T3[Network_3_T3.loc[:, 'EM_Status']=='Epithelial']).loc[:, 'BACH1_Status'].value_counts().sort_index()

In [ ]:
(Network_3_T2[Network_3_T2.loc[:, 'EM_Status']=='Epithelial']).loc[:, 'BACH1_Status'].value_counts().sort_index()

In [ ]:
(Network_3_T1[Network_3_T1.loc[:, 'EM_Status']=='Epithelial']).loc[:, 'BACH1_Status'].value_counts().sort_index()

In [ ]:
#summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'Epithelial', 'EM', 'BACH1_Status').loc[:,'Probabilty Given Epithelial']
'''conditional=pd.DataFrame( index=['BACH1_Positive','BACH1_Negative'], columns=['Given Epithelial'])
conditional.loc[:,'Given Epithelial']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'Epithelial', 'EM', 'BACH1_Status').loc[:,'Probabilty Given Epithelial']
conditional.loc[:,'Given Hybrid']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'Hybrid', 'EM', 'BACH1_Status').loc[:,'Probabilty Given Epithelial']
conditional.loc[:,'Given Epithelial']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'Epithelial', 'EM', 'BACH1_Status').loc[:,'Probabilty Given Epithelial']
conditional.loc[:,'Given Epithelial']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'Epithelial', 'EM', 'BACH1_Status').loc[:,'Probabilty Given Epithelial']
#conditional.loc[:,'Given BACH1 Negative']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Negative', 'BACH1', 'Stemness_Status').loc[:,'Probabilty Given BACH1_Negative']
#conditional.loc[:,'Given RKIP Positive']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Positive', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Positive']
#conditional.loc[:,'Given RKIP Negative']=summari_rev(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Negative']
conditional.T'''
#(Network_3_T1[Network_3_T1.loc[:, 'RKIP_Status']=='RKIP_Negative']).value_counts().sort_index()
'''conditional=pd.DataFrame( index=['Non-Stem-like','Stem-like'])
conditional.loc[:,'Given BACH1 Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Positive', 'BACH1', 'Stemness_Status').loc[:,'Probabilty Given BACH1_Positive']
conditional.loc[:,'Given BACH1 Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Negative', 'BACH1', 'Stemness_Status').loc[:,'Probabilty Given BACH1_Negative']
conditional.loc[:,'Given RKIP Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Positive', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Positive']
conditional.loc[:,'Given RKIP Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Negative']
conditional.T'''
'''conditional=pd.DataFrame( index=['Epithelial','Hybrid','Mesenchymal'])
conditional.loc[:,'Given BACH1 Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Positive', 'BACH1', 'EM_Status').loc[:,'Probabilty Given BACH1_Positive']
conditional.loc[:,'Given BACH1 Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Negative', 'BACH1', 'EM_Status').loc[:,'Probabilty Given BACH1_Negative']
conditional.loc[:,'Given RKIP Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Positive', 'RKIP', 'EM_Status').loc[:,'Probabilty Given RKIP_Positive']
conditional.loc[:,'Given RKIP Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'EM_Status').loc[:,'Probabilty Given RKIP_Negative']
conditional.T'''
'''summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Negative', 'BACH1', 'Stemness_Status')
summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Positive', 'BACH1', 'Stemness_Status')
summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Positive', 'RKIP', 'Stemness_Status')
summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'Stemness_Status')'''
'''conditional=pd.DataFrame( index=['Non-Stem-like','Stem-like',])
conditional.loc[:,'Given BACH1 Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Positive', 'BACH1', 'Stemness_Status').loc[:,'Probabilty Given BACH1_Positive']
conditional.loc[:,'Given BACH1 Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1_Negative', 'BACH1', 'Stemness_Status').loc[:,'Probabilty Given BACH1_Negative']
conditional.loc[:,'Given RKIP Positive']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Positive', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Positive']
conditional.loc[:,'Given RKIP Negative']=summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'Stemness_Status').loc[:,'Probabilty Given RKIP_Negative']
conditional'''
#summari(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP_Negative', 'RKIP', 'EM_Status')

In [ ]:
def summar(Network1,Network2,Network3, case1, given, given_col):
    cond=pd.DataFrame()
    cond.loc[:,'Trial 1']=((Network1[Network1.loc[:, given_col]==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False) /sum(((Network1[Network1.loc[:,'Stemness_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.loc[:,'Trial 2']=((Network2[Network2.loc[:,given_col ]==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False)/sum(((Network2[Network2.loc[:,'Stemness_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.loc[:,'Trial 3']=((Network3[Network3.loc[:,given_col]==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False)/sum(((Network3[Network3.loc[:,'Stemness_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.index=[case1 +'+ve', case1 + '-ve']
    cond.loc[:,'Mean'] = cond.iloc[:,0:3].mean(axis=1)
    cond.loc[:,'Standard Deviation'] = cond.iloc[:,0:3].std(axis=1)/cond.loc[:,'Mean']
    cond = cond*100
    cond=cond.round(2)
    cond.loc[:,'Probabilty Given '  + given] = cond.loc[:,'Mean'].astype(str) + ' ± ' + cond.loc[:,'Standard Deviation'].astype(str) + '%'
    return cond

In [ ]:
Network_3_T3

In [ ]:
summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Non-Stem-like','Stemness_Status' )

In [ ]:
summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like' ,'Stemness_Status' )

In [ ]:
summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Non-Stem-like' ,'Stemness_Status' )

In [ ]:
summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Stem-like','Stemness_Status'  )

In [ ]:
summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like' ,'Stemness_Status' ).loc['BACH1+ve','Probabilty Given Stem-like']
conditional

In [ ]:
pd.DataFrame([[0]*2]*4)

Figure 4E (ii)

In [ ]:
conditional=pd.DataFrame([[0]*2]*4, index=['BACH1+ve','BACH1-ve','RKIP+ve','RKIP-ve'])
conditional.columns=['Stem-like','Non-Stem-like']
conditional.loc['BACH1+ve','Stem-like',]=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like' ,'Stemness_Status' ).loc['BACH1+ve','Probabilty Given Stem-like']
conditional.loc['BACH1+ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Non-Stem-like','Stemness_Status'  ).loc['BACH1+ve','Probabilty Given Non-Stem-like']
conditional.loc['RKIP+ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Stem-like','Stemness_Status'  ).loc['RKIP+ve','Probabilty Given Stem-like']
conditional.loc['RKIP+ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Non-Stem-like' ,'Stemness_Status' ).loc['RKIP+ve','Probabilty Given Non-Stem-like']
conditional.loc['BACH1-ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like','Stemness_Status'  ).loc['BACH1-ve','Probabilty Given Stem-like']
conditional.loc['BACH1-ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Non-Stem-like','Stemness_Status'  ).loc['BACH1-ve','Probabilty Given Non-Stem-like']
conditional.loc['RKIP-ve','Stem-like'] = summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Stem-like','Stemness_Status'  ).loc['RKIP-ve','Probabilty Given Stem-like']
conditional.loc['RKIP-ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Non-Stem-like','Stemness_Status'  ).loc['RKIP-ve','Probabilty Given Non-Stem-like']
conditional.columns=['Given Stem-like','Given Non-Stem-like']
conditional

In [ ]:
'''conditional=pd.DataFrame([[0]*2]*4, columns=['Stem-like','Non-Stem-like'])
conditional.index=['BACH1+ve', 'BACH1-ve', 'RKIP+ve', 'RKIP-ve']
conditional.loc['BACH1+ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like' ).loc['BACH1+ve','Probabilty Given Stem-like']
conditional.loc['BACH1+ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Non-Stem-like' ).loc['BACH1+ve','Probabilty Given Non-Stem-like']
conditional.loc['RKIP+ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Stem-like' ).loc['RKIP+ve','Probabilty Given Stem-like']
conditional.loc['RKIP+ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Non-Stem-like' ).loc['RKIP+ve','Probabilty Given Non-Stem-like']
conditional.loc['BACH1-ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Stem-like' ).loc['BACH1-ve','Probabilty Given Stem-like']
conditional.loc['BACH1-ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Non-Stem-like' ).loc['BACH1-ve','Probabilty Given Non-Stem-like']
conditional.loc['RKIP-ve','Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Stem-like' ).loc['RKIP-ve','Probabilty Given Stem-like']
conditional.loc['RKIP-ve','Non-Stem-like']=summar(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Non-Stem-like' ).loc['RKIP-ve','Probabilty Given Non-Stem-like']
conditional.columns=['Given Stem-like','Given Non-Stem-like']
conditional'''

In [ ]:
def summary_EM(Network1,Network2,Network3, case1, given):
    cond=pd.DataFrame()
    cond.loc[:,'Trial 1']=((Network1[Network1.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False)/sum(((Network1[Network1.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.loc[:,'Trial 2']=((Network2[Network2.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False)/sum(((Network2[Network2.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.loc[:,'Trial 3']=((Network3[Network3.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False)/sum(((Network3[Network3.loc[:,'EM_Status']==given]).loc[:,case1]>0).value_counts().sort_index(ascending=False))
    cond.index=[case1 +'+ve', case1 + '-ve']
    cond.loc[:,'Mean'] = cond.iloc[:,0:3].mean(axis=1)
    cond.loc[:,'Standard Deviation'] = cond.iloc[:,0:3].std(axis=1)/cond.loc[:,'Mean']
    cond = cond*100
    cond=cond.round(2)
    cond.loc[:,'Probabilty Given '  + given] = cond.loc[:,'Mean'].astype(str) + ' ± ' + cond.loc[:,'Standard Deviation'].astype(str) + '%'
    return cond


In [ ]:
summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Epithelial' )

In [ ]:
summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Mesenchymal' )

Figure 4E (ii)

In [ ]:
conditional_EM=pd.DataFrame([[0]*3]*4)
conditional_EM.columns=['Epithelial','Hybrid','Mesenchymal']
conditional_EM.index=['BACH1+ve', 'BACH1-ve', 'RKIP+ve', 'RKIP-ve']
conditional_EM.loc['BACH1+ve','Epithelial'] = summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Epithelial' ).loc['BACH1+ve','Probabilty Given Epithelial']
conditional_EM.loc['BACH1+ve','Hybrid']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Hybrid' ).loc['BACH1+ve','Probabilty Given Hybrid']
conditional_EM.loc['BACH1+ve','Mesenchymal']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Mesenchymal' ).loc['BACH1+ve','Probabilty Given Mesenchymal']
conditional_EM.loc['BACH1-ve','Epithelial']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Epithelial' ).loc['BACH1-ve','Probabilty Given Epithelial']
conditional_EM.loc['BACH1-ve','Hybrid']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Hybrid' ).loc['BACH1-ve','Probabilty Given Hybrid']
conditional_EM.loc['BACH1-ve','Mesenchymal']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'BACH1' ,'Mesenchymal' ).loc['BACH1-ve','Probabilty Given Mesenchymal']
conditional_EM.loc['RKIP+ve','Epithelial']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Epithelial' ).loc['RKIP+ve','Probabilty Given Epithelial']
conditional_EM.loc['RKIP+ve','Hybrid']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Hybrid' ).loc['RKIP+ve','Probabilty Given Hybrid']
conditional_EM.loc['RKIP+ve','Mesenchymal']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Mesenchymal' ).loc['RKIP+ve','Probabilty Given Mesenchymal']
conditional_EM.loc['RKIP-ve','Epithelial']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Epithelial' ).loc['RKIP-ve','Probabilty Given Epithelial']
conditional_EM.loc['RKIP-ve','Hybrid']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Hybrid' ).loc['RKIP-ve','Probabilty Given Hybrid']
conditional_EM.loc['RKIP-ve','Mesenchymal']=summary_EM(Network_3_T3,Network_3_T2,Network_3_T1, 'RKIP' ,'Mesenchymal' ).loc['RKIP-ve','Probabilty Given Mesenchymal']
conditional_EM.columns=['Given Epithelial','Given Hybrid','Given Mesenchymal']
conditional_EM


In [ ]:
'''RKIP_summary_EM = pd.DataFrame([[0]]*3)
RKIP_summary_EM.loc[:,'Net3_T1']=np.array(cross(Network_3_T1, 'EM' , 'RKIP').iloc[:,-1])
RKIP_summary_EM.loc[:,'Net3_T2']=np.array(cross(Network_3_T2, 'EM' , 'RKIP').iloc[:,-1])
RKIP_summary_EM.loc[:,'Net3_T3']=np.array(cross(Network_3_T3, 'EM' , 'RKIP').iloc[:,-1])
RKIP_summary_EM= RKIP_summary_EM.loc[:,'Net3_T1':]
RKIP_summary_EM.index = cross(Network_3_T1, 'EM' , 'RKIP').iloc[:,-1].index'''

In [ ]:
'''RKIP_summary_EM.loc[:,'Mean'] =RKIP_summary_EM.loc[:, ['Net3_T1','Net3_T2','Net3_T3']].T.describe().loc['mean',:]
RKIP_summary_EM.loc[:,'Std'] =RKIP_summary_EM.loc[:, ['Net3_T1','Net3_T2','Net3_T3']].T.describe().loc['std',:]'''

In [ ]:
'''RKIP_summary_EM.loc[:,'printable']=round(RKIP_summary_EM.loc[:,'Mean'],2).astype(str) + ' ± ' + round(RKIP_summary_EM.loc[:,'Std'],2).astype(str) 
RKIP_summary_EM'''

In [ ]:
'''RKIP_summary_EM'''

In [ ]:
'''BACH1_summary_EM = pd.DataFrame([[0]]*3)
BACH1_summary_EM.loc[:,'Net3_T1']=np.array(cross(Network_3_T1, 'EM' , 'BACH1').iloc[:,-1])
BACH1_summary_EM.loc[:,'Net3_T2']=np.array(cross(Network_3_T2, 'EM' , 'BACH1').iloc[:,-1])
BACH1_summary_EM.loc[:,'Net3_T3']=np.array(cross(Network_3_T3, 'EM' , 'BACH1').iloc[:,-1])
BACH1_summary_EM= BACH1_summary_EM.loc[:,'Net3_T1':]
BACH1_summary_EM.index = cross(Network_3_T1, 'EM' , 'BACH1').iloc[:,-1].index'''

In [ ]:
'''BACH1_summary_EM.loc[:,'Mean'] =BACH1_summary_EM.loc[:, ['Net3_T1','Net3_T2','Net3_T3']].T.describe().loc['mean',:]
BACH1_summary_EM.loc[:,'Std'] =BACH1_summary_EM.loc[:, ['Net3_T1','Net3_T2','Net3_T3']].T.describe().loc['std',:]
BACH1_summary_EM'''

In [ ]:
'''BACH1_summary_EM'''

In [ ]:
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [ ]:
pca = PCA(n_components=10)

In [ ]:
Network_3_T3.loc[:,['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']]=pca.fit_transform(Network_3_T3.loc[:,'ERa66':'OCT4'])
Network_3_T3

In [ ]:
def hue_scatterplot(dataframe , x, y , x_label, y_label, hue, colors='viridis', vmax = 3, vmin =-3):
    plt.figure(figsize=(10,6))
    #plt.rcParams['figure.dpi']= 1000
    #plt.title('Network 1')
    #ax.figsize = (19, 8)
    
    #fig, ax = plt.subplots()
    plo = plt.scatter(data= dataframe, x=x, y=y, s = 0.5,  c = dataframe.loc[:,hue] , cmap = colors, vmax = vmax, vmin =vmin)
    plt.xlabel(x_label, fontsize=17)
    plt.ylabel(y_label,  fontsize=17)
    plt.xticks(size=17)
    plt.yticks(size=17)

    plt.legend('')
    
    
    cbar  = plt.colorbar(plo)
    cbar.ax.tick_params(labelsize=17)
    cbar.set_label(hue, rotation=270, size = 17, labelpad= 14)



Figure 3E (i) & (ii)

In [ ]:
Explained_vaience = pd.DataFrame(pca.explained_variance_ratio_,  index = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10'])*100
Explained_vaience.columns = ['Pecentage Explained Vaience']
Explained_vaience

Figure 3 E (i)

In [ ]:
hue_scatterplot(Network_3_T3, 'PC1', 'PC2','PC1 (48.8% of the Total Varience )', 'PC2 (19.5% of the Total Varience )', hue='RKIP' ,  colors='seismic', vmax = 3, vmin =-3)
plt.title("RACIPE solutions shaded based on RKIP expression", size=19, fontname='Arial', fontweight='bold')


In [ ]:
hue_scatterplot(Network_3_T3, 'PC1', 'PC2', 'PC1 (48.8% of the Total Varience )', 'PC2 (19.5% of the Total Varience )', hue='BACH1' ,  colors='seismic', vmax = 3, vmin =-3)
plt.title("RACIPE solutions shaded based on BACH1 expression", size=19, fontname='Arial', fontweight='bold')

Figure 3 B

In [ ]:
Topo_Network_5 = pd.read_csv('Network5_trial2.topo', sep='\t')
Topo_Network_5

In [ ]:
def Topo_adj(dp):
    n=len(set(dp['Source']))
    adj = pd.DataFrame([[0]*n]*n)
    adj.columns=set(dp.loc[:,'Source'])
    adj.index=set(dp.loc[:,'Source'])
    for i in range(len(dp)):
        if dp.iloc[i,2] == 1:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = 1
        elif dp.iloc[i,2] == 2:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = -1

    return adj


In [ ]:
Topo_adj(Topo_Network_5)

Figure 3B Adjacency Matrix

In [ ]:
#inf = influence(Topo_Network_5, 10)
sns.set(font_scale=1.1)
plt.rcParams['figure.dpi']= 1000

#plt.xlabel(fontsize=16);
#plt.ylabel(fontsize=16);
#plt.title('Sales Data', fontsize=20)
with plt.style.context({
                        'xtick.labelsize':16,
                        'ytick.labelsize':16}):
        sns.clustermap(Topo_adj(Topo_Network_5), annot = True, cmap ='coolwarm',
                       linecolor ='black', linewidths = 1)
plt.tick_params(axis='both', which='major', labelsize=13)
#plt.ylabel('Source', fontsize=16)

In [ ]:
def influence(dp, j):
    n=len(set(dp['Source']))
    infl = pd.DataFrame([[0]*n]*n)
    infl.columns=set(dp.loc[:,'Source'])
    infl.index=set(dp.loc[:,'Source'])
    adj = Topo_adj(dp)

    maxi = abs(adj)
    pr = adj
    mr = abs(adj)
    infl = adj
    for i in range(1, j):
        pr=adj@pr
        mr = maxi@mr
        qwe = pr/mr

        infl = infl + qwe.replace(np.nan, 0)
        
        

    infl  = infl/j
    infl=infl.round(2)
    return infl

In [ ]:
plt.figure(figsize = (15,8))

plt.rcParams['figure.dpi']= 1000
with plt.style.context({
                        'xtick.labelsize':16,
                        'ytick.labelsize':16}):
        sns.clustermap(influence(Topo_Network_5, 8), annot = True, cmap ='coolwarm', vmax = 1, vmin =-1,
                       linecolor ='black', linewidths = 1)
        
plt.tick_params(axis='both', which='major', labelsize=13)

Figure 3 C Pathlength 8

In [ ]:
plt.figure(figsize = (15,8))

plt.rcParams['figure.dpi']= 1000
with plt.style.context({
                        'xtick.labelsize':16,
                        'ytick.labelsize':16}):
        sns.clustermap(influence(Topo_Network_5, 8),  cmap ='coolwarm', vmax = 1, vmin =-1,
                       linecolor ='black', linewidths = 1)
        
plt.tick_params(axis='both', which='major', labelsize=13)


In [ ]:
tup=influence(Topo_Network_5, 8).sort_values(by='miR200', ascending=True)
tupi=tup.T.sort_values(by='miR200', ascending=True)
tupi

In [ ]:
'''plt.figure(figsize=(20,20))
sns.heatmap(tupi, linewidths = 1,  cmap="coolwarm", square = True, fmt="",vmax = 0.8, vmin =-0.8, annot = False
            ,cbar_kws={'orientation': 'horizontal', 'pad' : 0.13, 'aspect': 40}, annot_kws={'size': 15} )
plt.tick_params(axis='both', which='major', labelsize=16)'''

Identifying Teams

In [ ]:
Team1 = []
Team2 = []
infl = influence(Topo_Network_5, 8)
for i in infl.index:
    if infl.loc[i,'miR200'] >= 0:
        Team1.append(i)
    elif infl.loc[i,'miR200'] <= 0:
        Team2.append(i) 

print(Team1)
print(Team2)

In [ ]:
TeamScores = pd.DataFrame(0, columns = [0,1], index = [0,1] )

teams = [Team1, Team2]

for t1 in range(len(teams)):
    for t2 in range(len(teams)):
        Tsc= 0 
        count = 0
        #print(teams[t1])
        for i in teams[t1]:
            for j in teams[t2]:
                 Tsc += tupi.loc[i,j]
                 count +=1
                 #print(t1,t2)
        TeamScores.loc[t1,t2] = Tsc/count   

TeamScores.columns = ['TeamB','TeamA']

TeamScores.index = ['TeamB','TeamA']
TeamScores

Team Strength og Wild Type Team

In [ ]:
abs(TeamScores).sum(axis = 1).sum(axis = 0)/4

In [ ]:
import random
log = []
dp=Topo_Network_5


for oi in range(100):
    edges = list(dp['Type'])
    random.shuffle(edges)
    dp['Type'] = edges

    n=len(set(dp['Source']))
    adj = pd.DataFrame([[0]*n]*n)
    adj.columns=set(dp.loc[:,'Source'])
    adj.index=set(dp.loc[:,'Source'])
    
    for i in range(len(dp)):
        if dp.iloc[i,2] == 1:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = 1
        elif dp.iloc[i,2] == 2:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = -1


    n=len(set(dp['Source']))
    infl = pd.DataFrame([[0]*n]*n)
    infl.columns=set(dp.loc[:,'Source'])
    infl.index=set(dp.loc[:,'Source'])
    
    maxi = abs(adj)
    pr = adj
    mr = abs(adj)
    infl = adj
    for i in range(1, 8):
        pr=adj@pr
        mr = maxi@mr
        qwe = pr/mr

        infl = infl + qwe.replace(np.nan, 0)
        
    infl  = infl/8
    infl


    Team1 = []
    Team2 = []

    for i in infl.index:
        if infl.loc[i,'miR200'] >= 0:
            Team1.append(i)
        elif infl.loc[i,'miR200'] <= 0:
            Team2.append(i) 
    

    TeamScores = pd.DataFrame(0, columns = [0,1], index = [0,1] )

    teams = [Team1, Team2]

    for t1 in range(len(teams)):
        for t2 in range(len(teams)):
            Tsc= 0 
            count = 1
            #print(teams[t1])
            for i in teams[t1]:
                for j in teams[t2]:
                    Tsc += infl.loc[i,j]
                    count +=1
                    #print(t1,t2)
            TeamScores.loc[t1,t2] = Tsc/count   

    TeamScores.columns = ['TeamB','TeamA']

    TeamScores.index = ['TeamB','TeamA']
    TeamScores
    
    log.append(abs(TeamScores).sum(axis = 1).sum(axis = 0)/4)



In [ ]:
qwer = pd.DataFrame(log)
qwer[1]= qwer.index

In [ ]:
sns.histplot(data=qwer, x = 0,stat = "probability", bins = 50,palette="tab10",element="step")
plt.axvline(x = 0.37318735827664407, color = 'r', linestyle = '-')
plt.xlabel('Team Strength')
plt.ylabel('Frequency')

In [ ]:
import random
log = []
dp=Topo_Network_5


for oi in range(1000):
    edges = list(dp['Type'])
    random.shuffle(edges)
    dp['Type'] = edges

    n=len(set(dp['Source']))
    adj = pd.DataFrame([[0]*n]*n)
    adj.columns=set(dp.loc[:,'Source'])
    adj.index=set(dp.loc[:,'Source'])
    
    for i in range(len(dp)):
        if dp.iloc[i,2] == 1:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = 1
        elif dp.iloc[i,2] == 2:
            adj.loc[dp.iloc[i,0] , dp.iloc[i,1] ] = -1


    n=len(set(dp['Source']))
    infl = pd.DataFrame([[0]*n]*n)
    infl.columns=set(dp.loc[:,'Source'])
    infl.index=set(dp.loc[:,'Source'])
    
    maxi = abs(adj)
    pr = adj
    mr = abs(adj)
    infl = adj
    for i in range(1, 8):
        pr=adj@pr
        mr = maxi@mr
        qwe = pr/mr

        infl = infl + qwe.replace(np.nan, 0)
        
    infl  = infl/8
    infl


    Team1 = []
    Team2 = []

    for i in infl.index:
        if infl.loc[i,'miR200'] >= 0:
            Team1.append(i)
        elif infl.loc[i,'miR200'] <= 0:
            Team2.append(i) 
    

    TeamScores = pd.DataFrame(0, columns = [0,1], index = [0,1] )

    teams = [Team1, Team2]

    for t1 in range(len(teams)):
        for t2 in range(len(teams)):
            Tsc= 0 
            count = 1
            #print(teams[t1])
            for i in teams[t1]:
                for j in teams[t2]:
                    Tsc += infl.loc[i,j]
                    count +=1
                    #print(t1,t2)
            TeamScores.loc[t1,t2] = Tsc/count   

    TeamScores.columns = ['TeamB','TeamA']

    TeamScores.index = ['TeamB','TeamA']
    TeamScores
    
    log.append(abs(TeamScores).sum(axis = 1).sum(axis = 0)/4)



In [ ]:
qwer = pd.DataFrame(log)
qwer[1]= qwer.index

Figure 3D

In [ ]:
plt.rcParams['figure.dpi']= 1000
sns.histplot(data=qwer, x = 0,stat = "probability", bins = 50,palette="tab10",element="step")
plt.axvline(x = 0.37318735827664407, color = 'r', linestyle = '-', label = 'WT'+'TS='+str(0.286))
plt.text(0.352, 0.025, 'Team Strength ( WT Network )', size='12', color='Red', rotation=90)
plt.xlabel('Team Strength', fontweight='bold', fontsize='15' , fontname='Arial')
plt.ylabel('Frequency', fontweight='bold', fontsize='15' , fontname='Arial')
plt.xticks( fontsize='13')
plt.yticks( fontsize='13')



#plt.figure(figsize=(30,20))
